# Import libraries

In [2]:
import pandas as pd
import os
import numpy as np
import datetime as dt

# Collection of data files

## Europe Data

### Load in the Data

In [294]:
# Read Europe Data 
## Director
Europe_Director_Education = pd.read_excel(
    "Europe/Europe - Director Profile - Education & Achievements.xlsx")
Europe_Director_Characteristics = pd.read_excel(
    "Europe/Europe - Director Profile - Characteristics.xlsx", header=[0,1])
Europe_Director_Employment_Current_Board = pd.read_excel(
       "Europe/Europe - Director Profile - Employment Current Board.xlsx")
Europe_Director_Employment_Current_NonBoard = pd.read_excel(
    "Europe/Europe - Director Profile - Employment Current NonBoard.xlsx")
Europe_Director_Employment_Historical_Board_1 = pd.read_excel(
    "Europe/Europe - Director Profile - Employment Historical Board - 1.xlsx")
Europe_Director_Employment_Historical_Board_2 = pd.read_excel(
    "Europe/Europe - Director Profile - Employment Historical Board - 2.xlsx")
Europe_Director_Employment_Historical_NonBoard = pd.read_excel(
    "Europe/Europe - Director Profile - Employment Historical NonBoard.xlsx")

## SMDE
Europe_SMDEs_Education = pd.read_excel(
    "Europe/Europe - SMDEs Profile - Education & Achievements.xlsx")
Europe_SMDEs_Characteristics_1 = pd.read_excel(
       "Europe/Europe - SMDEs Profile - Characteristics - 1.xlsx", header=[0,1])
Europe_SMDEs_Characteristics_2 = pd.read_excel(
    "Europe/Europe - SMDEs Profile - Characteristics - 2.xlsx", header=[0,1])
Europe_SMDEs_Employment_Current_Board = pd.read_excel(
       "Europe/Europe - SMDEs Profile - Employment Current Board.xlsx")
Europe_SMDEs_Employment_Current_NonBoard = pd.read_excel(
       "Europe/Europe - SMDEs Profile - Employment Current NonBoard.xlsx")
Europe_SMDEs_Employment_Historical_Board = pd.read_excel(
    "Europe/Europe - SMDEs Profile - Employment Historical Board.xlsx")
Europe_SMDEs_Employment_Historical_NonBoard_1 = pd.read_excel(
    "Europe/Europe - SMDEs Profile - Employment Historical NonBoard - 1.xlsx")
Europe_SMDEs_Employment_Historical_NonBoard_2 = pd.read_excel(
    "Europe/Europe - SMDEs Profile - Employment Historical NonBoard - 2.xlsx")


### Stacking the File Multiples (Characteristics, SMDEs Historical NonBoard)

Stacking the file multiples using `pd.concat` to get a full array of the data.

Concatenating the same Directors and SMDEs dataframes together to reduce the numbr of dataframes to use

In [385]:
Europe_Director_Employment_Historical_Board = pd.concat([Europe_Director_Employment_Historical_Board_1, Europe_Director_Employment_Historical_Board_2])
Europe_SMDEs_Characteristics = pd.concat([Europe_SMDEs_Characteristics_1, Europe_SMDEs_Characteristics_2])
Europe_SMDEs_Employment_Historical_NonBoard = pd.concat([Europe_SMDEs_Employment_Historical_NonBoard_1, Europe_SMDEs_Employment_Historical_NonBoard_2])


In [386]:
Europe_Education = pd.concat([Europe_Director_Education,Europe_SMDEs_Education])
Europe_Employment_Historical_Board = pd.concat([Europe_Director_Employment_Historical_Board, Europe_SMDEs_Employment_Historical_Board], ignore_index=True)
Europe_Employment_Current_Board = pd.concat([Europe_Director_Employment_Current_Board, Europe_SMDEs_Employment_Current_Board], ignore_index=True)
Europe_Employment_Historical_NonBoard = pd.concat([Europe_Director_Employment_Historical_NonBoard, Europe_SMDEs_Employment_Historical_NonBoard], ignore_index=True)
Europe_Employment_Current_NonBoard = pd.concat([Europe_Director_Employment_Current_NonBoard, Europe_SMDEs_Employment_Current_NonBoard], ignore_index=True)
Europe_Employment_Characteristics = pd.concat([Europe_Director_Characteristics, Europe_SMDEs_Characteristics], ignore_index=True)


Selecting just rows with Ticker: which represents that the company is public (our main scope of investigation)

In [387]:
# Select the companies where the Ticker is not null in all the files -> we just want public listed companies
Europe_Employment_Historical_Board = Europe_Employment_Historical_Board[Europe_Employment_Historical_Board['Company Ticker'].notnull()].reset_index(drop=True)
Europe_Employment_Current_Board = Europe_Employment_Current_Board[Europe_Employment_Current_Board['Company Ticker'].notnull()].reset_index(drop=True)
Europe_Employment_Historical_NonBoard = Europe_Employment_Historical_NonBoard[Europe_Employment_Historical_NonBoard['Company Ticker'].notnull()].reset_index(drop=True)
Europe_Employment_Current_NonBoard = Europe_Employment_Current_NonBoard[Europe_Employment_Current_NonBoard['Company Ticker'].notnull()].reset_index(drop=True)

Change the column name of ED/NED of historical and current board dataframes into ED/NED/SM for the concat process afterwards

In [388]:
Europe_Employment_Historical_Board.rename(columns={'ED/NED': 'ED/NED/SM'}, inplace=True)
Europe_Employment_Current_Board.rename(columns={'ED/NED': 'ED/NED/SM'}, inplace=True)


In [389]:
# Merge all the dataframes related to employees into one
Europe_lst = [Europe_Employment_Historical_Board, Europe_Employment_Current_Board,Europe_Employment_Historical_NonBoard, Europe_Employment_Current_NonBoard]
Europe_Employment = pd.concat(Europe_lst, ignore_index=True)

### Employment Data cleaning

In [390]:
Europe_Employment.duplicated().sum()

81234

Remove the duplicates in the dataframes

In [391]:
# Remove the duplicates from the Employment
Europe_Employment.drop_duplicates(inplace=True)

### Directors' characteristics cleaning

In [392]:
Europe_Employment_Characteristics = Europe_Employment_Characteristics.loc[:, ['Unnamed: 0_level_0', 'Unnamed: 1_level_0','Unnamed: 5_level_0', 'Unnamed: 6_level_0']]
Europe_Employment_Characteristics = Europe_Employment_Characteristics.droplevel(0, axis=1)  # drop the first level of the df


In [393]:
Europe_Employment_Characteristics.drop_duplicates(inplace=True, ignore_index=True)

In [394]:
Europe_Employment_Characteristics

,Director Name,DirectorID*,Gender,Nationality
0,A (Ab) F van der Touw,206397712886,M,Dutch
1,A (Ab) J Pasman,11345263500,M,Dutch
2,A (Allard) Van Spaandonk,24695787865,M,Dutch
3,A (Angelique) A Huijben-Pijnenburg,11711154218,F,Dutch
4,A (Anja) D Vijselaar,25356448525,F,Dutch
...,...,...,...,...
168170,Ziya Engin Tuncay,8454821006,M,British
168171,Zlata Bakk Kovacevic,8487972396,F,Unknown
168172,Zwier Jan van Puijenbroek,8830027652,M,Dutch
168173,Zygmunt Mackało,94474112746,M,Unknown


In [ ]:
# Merge the Employment and Director Characteristics by DirectorID*
Europe_df = pd.merge(Europe_Employment, Europe_Employment_Characteristics, on='DirectorID*', how='left')

# Drop Director Name_y column
Europe_df.drop(['Director Name_y'], axis=1, inplace=True)
Europe_df

In [397]:
Europe_df

,Country,DirectorID*,Director Name_x,CompanyID*,Company Name,Company ISIN,Company Ticker,Company Index,Sector Name,Company Type,Role,Role Description,ED/NED/SM,Start Date,End Date,Committee Name,Committee Role,Gender,Nationality
0,Netherlands,11345263500,A (Ab) J Pasman,17593156,ACCELL GROUP NV,NL0009767532,ACCEL,NaN,Leisure Goods,Quoted,Independent Chairman,NaN,NED,2010-04-22 00:00:00,2019-04-24 00:00:00,Remuneration and Selection,Member,M,Dutch
1,Netherlands,11345263500,A (Ab) J Pasman,17593156,ACCELL GROUP NV,NL0009767532,ACCEL,NaN,Leisure Goods,Quoted,Independent Director,NaN,NED,2019-04-24 00:00:00,2019-12-31 00:00:00,Remuneration and Selection,Member,M,Dutch
2,Austria,51690811635,A (Aris) G Bogdaneris,8489622427,RAIFFEISEN BANK INTERNATIONAL AG (Raiffeisen I...,AT0000606306,RBI,ATX,Banks,Quoted,Board Member - ED,Retail Banking,ED,2004-10-01 00:00:00,2010-12-01 00:00:00,NaN,NaN,M,Canadian
3,Austria,51690811635,A (Aris) G Bogdaneris,8489622427,RAIFFEISEN BANK INTERNATIONAL AG (Raiffeisen I...,AT0000606306,RBI,ATX,Banks,Quoted,COO,Responsible for Consumer Banking Group & Austr...,ED,2010-12-01 00:00:00,2015-03-31 00:00:00,NaN,NaN,M,Canadian
4,Netherlands,3590756873,A (Dries) B Van Luyk,13653543,AALBERTS NV (Aalberts Industries NV prior to 0...,NL0000852564,AALB,AEX MID-CAP,Engineering & Machinery,Quoted,Board Member - SD,NaN,NED,1996,2007-04-23 00:00:00,NaN,NaN,M,Dutch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552385,Slovenia,180805111015,Zvone Simončič,13188656371,KRKA DD,SI0031102120,KRKG,NaN,Pharmaceuticals and Biotechnology,Quoted,Director - Pharmaceuticals,Research and Development,SM,2016-01-01 00:00:00,NaN,NaN,NaN,M,Unknown
552386,Germany,196117712164,Zyad Neil Benaissa,168431910018,STRATEC SE (STRATEC Biomedical AG prior to 12/...,DE000STRA555,SBS,CDAX,Health,Quoted,Sales Director,NaN,SM,2019-04-01 00:00:00,NaN,NaN,NaN,M,Unknown
552387,Norway,15954139247,Zygimantas Dirse,2468311452,KITRON ASA (Sonec ASA prior to 08/2000),NO0003079709,KIT,NaN,Electronic & Electrical Equipment,Quoted,Division MD,Kitron Electronics Manufacturing (Ningbo) Co Ltd,SM,2018,NaN,NaN,NaN,M,Unknown
552388,Denmark,15117668466,Zygmunt Chyla,395853456,AGAT EJENDOMME A/S (TK Development A/S prior t...,DK0010258995,AGAT,NaN,Real Estate,Quoted,Regional President/Regional Senior VP,Poland Initially Served as Divisional Director,SM,UnKnown,NaN,NaN,NaN,M,Unknown


## NA Data

### Load in the Data

In [238]:
# Read NA Data 
## Director
NA_Director_Education = pd.read_excel("NA/NA - Director Profile - Education & Achievements.xlsx")
NA_Director_Characteristics_1 = pd.read_excel(
       "NA/NA - Director Profile - Characteristics - 1.xlsx", header=[0,1])
NA_Director_Characteristics_2 = pd.read_excel(
    "NA/NA - Director Profile - Characteristics - 2.xlsx", header=[0, 1])
NA_Director_Employment_Current_Board = pd.read_excel(
    "NA/NA - Director Profile - Employment Current Board.xlsx")
NA_Director_Employment_Current_NonBoard = pd.read_excel(
       "NA/NA - Director Profile - Employment Current NonBoard.xlsx")
NA_Director_Employment_Historical_Board_1 = pd.read_excel(
       "NA/NA - Director Profile - Employment Historical Board - 1.xlsx")
NA_Director_Employment_Historical_Board_2 = pd.read_excel(
       "NA/NA - Director Profile - Employment Historical Board - 2.xlsx")
NA_Director_Employment_Historical_NonBoard_1 = pd.read_excel(
       "NA/NA - Director Profile - Employment Historical NonBoard - 1.xlsx")
NA_Director_Employment_Historical_NonBoard_2 = pd.read_excel(
    "NA/NA - Director Profile - Employment Historical NonBoard - 2.xlsx")

## SMDE
NA_SMDEs_Education_1 = pd.read_excel("NA/NA - SMDEs Profile - Education & Achievements - 1.xlsx")
NA_SMDEs_Education_2 = pd.read_excel("NA/NA - SMDEs Profile - Education & Achievements - 2.xlsx")
NA_SMDEs_Education_3 = pd.read_excel("NA/NA - SMDEs Profile - Education & Achievements.xlsx")     
NA_SMDEs_Characteristics_1 = pd.read_excel("NA/NA - SMDEs Profile - Characteristics - 1.xlsx", header=[0, 1])
NA_SMDEs_Characteristics_2 = pd.read_excel("NA/NA - SMDEs Profile - Characteristics - 2.xlsx", header=[0, 1])
NA_SMDEs_Characteristics_3 = pd.read_excel("NA/NA - SMDEs Profile - Characteristics - 3.xlsx", header=[0, 1])
NA_SMDEs_Characteristics_4 = pd.read_excel("NA/NA - SMDEs Profile - Characteristics - 4.xlsx", header=[0, 1])
NA_SMDEs_Characteristics_5 = pd.read_excel("NA/NA - SMDEs Profile - Characteristics - 5.xlsx", header=[0, 1])
NA_SMDEs_Employment_Current_Board = pd.read_excel(
       "NA/NA - SMDEs Profile - Employment Current Board.xlsx")
NA_SMDEs_Employment_Current_NonBoard = pd.read_excel(
       "NA/NA - SMDEs Profile - Employment Current NonBoard.xlsx")
NA_SMDEs_Employment_Historical_Board = pd.read_excel(
       "NA/NA - SMDEs Profile - Employment Historical Board.xlsx")
NA_SMDEs_Employment_Historical_NonBoard_1 = pd.read_excel(
       "NA/NA - SMDEs Profile - Employment Historical NonBoard - 1.xlsx")
NA_SMDEs_Employment_Historical_NonBoard_2 = pd.read_excel(
       "NA/NA - SMDEs Profile - Employment Historical NonBoard - 2.xlsx")
NA_SMDEs_Employment_Historical_NonBoard_3 = pd.read_excel(
    "NA/NA - SMDEs Profile - Employment Historical NonBoard - 2.xlsx")
NA_SMDEs_Employment_Historical_NonBoard_4 = pd.read_excel(
       "NA/NA - SMDEs Profile - Employment Historical NonBoard - 2.xlsx")
NA_SMDEs_Employment_Historical_NonBoard_5 = pd.read_excel(
    "NA/NA - SMDEs Profile - Employment Historical NonBoard - 2.xlsx")


### Stacking the File Multiples

In [398]:
NA_Director_Characteristics = pd.concat([NA_Director_Characteristics_1, NA_Director_Characteristics_2])
NA_Director_Employment_Historical_Board = pd.concat(
    [NA_Director_Employment_Historical_Board_1, NA_Director_Employment_Historical_Board_2])
NA_Director_Employment_Historical_NonBoard = pd.concat(
    [NA_Director_Employment_Historical_NonBoard_1, NA_Director_Employment_Historical_NonBoard_2])
NA_SMDEs_Education = pd.concat([NA_SMDEs_Education_1, NA_SMDEs_Education_2, NA_SMDEs_Education_3])
NA_SMDEs_Characteristics = pd.concat([NA_SMDEs_Characteristics_1, NA_SMDEs_Characteristics_2, NA_SMDEs_Characteristics_3, 
                                      NA_SMDEs_Characteristics_4, NA_SMDEs_Characteristics_5])
NA_SMDEs_Employment_Historical_NonBoard = pd.concat([NA_SMDEs_Employment_Historical_NonBoard_1, NA_SMDEs_Employment_Historical_NonBoard_2,
                                                     NA_SMDEs_Employment_Historical_NonBoard_3, NA_SMDEs_Employment_Historical_NonBoard_4,
                                                     NA_SMDEs_Employment_Historical_NonBoard_5])


In [399]:
NA_Education = pd.concat([NA_Director_Education, NA_SMDEs_Education])
NA_Employment_Historical_Board = pd.concat(
    [NA_Director_Employment_Historical_Board, NA_SMDEs_Employment_Historical_Board], ignore_index=True)
NA_Employment_Current_Board = pd.concat(
    [NA_Director_Employment_Current_Board, NA_SMDEs_Employment_Current_Board], ignore_index=True)
NA_Employment_Historical_NonBoard = pd.concat(
    [NA_Director_Employment_Historical_NonBoard, NA_SMDEs_Employment_Historical_NonBoard], ignore_index=True)
NA_Employment_Current_NonBoard = pd.concat(
    [NA_Director_Employment_Current_NonBoard, NA_SMDEs_Employment_Current_NonBoard], ignore_index=True)
NA_Employment_Characteristics = pd.concat(
    [NA_Director_Characteristics, NA_SMDEs_Characteristics], ignore_index=True)


Selecting just rows with Ticker: which represents that the company is public (our main scope of investigation)

In [400]:
# Select the companies where the Ticker is not null in all the files -> we just want public listed companies
NA_Employment_Historical_Board = NA_Employment_Historical_Board[NA_Employment_Historical_Board['Company Ticker'].notnull()].reset_index(drop=True)
NA_Employment_Current_Board = NA_Employment_Current_Board[NA_Employment_Current_Board['Company Ticker'].notnull()].reset_index(drop=True)
NA_Employment_Historical_NonBoard = NA_Employment_Historical_NonBoard[NA_Employment_Historical_NonBoard['Company Ticker'].notnull()].reset_index(drop=True)
NA_Employment_Current_NonBoard = NA_Employment_Current_NonBoard[NA_Employment_Current_NonBoard['Company Ticker'].notnull()].reset_index(drop=True)


Change the column name of ED/NED of historical and current board dataframes into ED/NED/SM for the concat process afterwards

In [401]:
# Change the Column names to be consistent with the other files
NA_Employment_Historical_Board.rename(columns={'ED/NED': 'ED/NED/SM'}, inplace=True)
NA_Employment_Current_Board.rename(columns={'ED/NED': 'ED/NED/SM'}, inplace=True)

In [402]:
# Merge all the dataframes related to employees into one
NA_lst = [NA_Employment_Historical_Board, NA_Employment_Current_Board,
              NA_Employment_Historical_NonBoard, NA_Employment_Current_NonBoard]
NA_Employment = pd.concat(NA_lst, ignore_index=True)


### Employment Data cleaning

In [403]:
NA_Employment.duplicated().sum()


547413

Remove the duplicates in the dataframes

In [404]:
NA_Employment.drop_duplicates(inplace=True)

### Directors' Characteristics cleaning

In [405]:
NA_Employment_Characteristics = NA_Employment_Characteristics.loc[:, [
    'Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 5_level_0', 'Unnamed: 6_level_0']]
NA_Employment_Characteristics = NA_Employment_Characteristics.droplevel(
    0, axis=1)  # drop the first level of the df


In [407]:
NA_Employment_Characteristics.drop_duplicates(inplace=True, ignore_index=True)

In [408]:
NA_Employment_Characteristics

,Director Name,DirectorID*,Gender,Nationality
0,. Aminbuhe,16594329810,M,Unknown
1,. Dalanguerban,23726336790,M,Unknown
2,A (Andy) Andrew Levison,29813210847,M,American
3,A (Bill) William Schenck III,13732157014,M,Unknown
4,A (Bob) Robert Pisano,20344911628,M,Unknown
...,...,...,...,...
517097,Zsoka Taylor McDonald,6956209411,M,Unknown
517098,Zubin G Malkani,7948969080,M,Unknown
517099,Zuher Ladak,81138110481,M,Unknown
517100,Zvi Avni,7982899383,M,Unknown


Merge NA_Employment with NA_Employment_Characteristics

In [409]:
# Merge the dataframes with the characteristics of the employees
NA_df = pd.merge(NA_Employment, NA_Employment_Characteristics,
                     on='DirectorID*', how='left')

# Drop Director Name_y column
NA_df.drop(['Director Name_y'], axis=1, inplace=True)
NA_df

## UK Data

### Load in the Data

In [249]:
# Read all the UK files
## Director
UK_Director_Education = pd.read_excel('UK/UK - Director Profile - Education & Achievements.xlsx')
UK_Director_Characteristics = pd.read_excel(
    'UK/UK - Director Profile - Characteristics.xlsx', header=[0, 1])
UK_Director_Employment_Current_Board = pd.read_excel('UK/UK - Director Profile - Employment Current Board.xlsx')

UK_Director_Employment_Current_NonBoard = pd.read_excel(
    'UK/UK - Director Profile - Employment Current NonBoard.xlsx')
UK_Director_Employment_Historical_Board = pd.read_excel(
    'UK/UK - Director Profile - Employment Historical Board.xlsx')
UK_Director_Employment_Historical_NonBoard = pd.read_excel(
    'UK/UK - Director Profile - Employment Historical NonBoard.xlsx')


## SMDE
UK_SMDEs_Education = pd.read_excel("UK/UK - SMDEs Profile - Education & Achievements.xlsx")
UK_SMDEs_Characteristics = pd.read_excel(
    'UK/UK - SMDEs Profile - Characteristics.xlsx', header=[0, 1])
UK_SMDEs_Employment_Current_Board = pd.read_excel(
    'UK/UK - SMDEs Profile - Employment Current Board.xlsx')
UK_SMDEs_Employment_Current_NonBoard = pd.read_excel(
    'UK/UK - SMDEs Profile - Employment Current NonBoard.xlsx')
UK_SMDEs_Employment_Historical_Board = pd.read_excel(
    'UK/UK - SMDEs Profile - Employment Historical Board.xlsx')
UK_SMDEs_Employment_Historical_NonBoard = pd.read_excel(
    'UK/UK - SMDEs Profile - Employment Historical NonBoard.xlsx')


### Stacking the File Multiples

In [414]:
UK_Education = pd.concat([UK_Director_Education, UK_SMDEs_Education])
UK_Employment_Historical_Board = pd.concat(
    [UK_Director_Employment_Historical_Board, UK_SMDEs_Employment_Historical_Board], ignore_index=True)
UK_Employment_Current_Board = pd.concat(
    [UK_Director_Employment_Current_Board, UK_SMDEs_Employment_Current_Board], ignore_index=True)
UK_Employment_Historical_NonBoard = pd.concat(
    [UK_Director_Employment_Historical_NonBoard, UK_SMDEs_Employment_Historical_NonBoard], ignore_index=True)
UK_Employment_Current_NonBoard = pd.concat(
    [UK_Director_Employment_Current_NonBoard, UK_SMDEs_Employment_Current_NonBoard], ignore_index=True)
UK_Employment_Characteristics = pd.concat(
    [UK_Director_Characteristics, UK_SMDEs_Characteristics], ignore_index=True)


Selecting just rows with Ticker: which represents that the company is public (our main scope of investigation)

In [415]:
# Select the companies where the Ticker is not null in all the files -> we just want public listed companies
UK_Employment_Historical_Board = UK_Employment_Historical_Board[UK_Employment_Historical_Board['Company Ticker'].notnull(
)].reset_index(drop=True)
UK_Employment_Current_Board = UK_Employment_Current_Board[UK_Employment_Current_Board['Company Ticker'].notnull(
)].reset_index(drop=True)
UK_Employment_Historical_NonBoard = UK_Employment_Historical_NonBoard[
    UK_Employment_Historical_NonBoard['Company Ticker'].notnull()].reset_index(drop=True)
UK_Employment_Current_NonBoard = UK_Employment_Current_NonBoard[UK_Employment_Current_NonBoard['Company Ticker'].notnull(
)].reset_index(drop=True)


Change the column name of ED/NED of historical and current board dataframes into ED/NED/SM for the concat process afterwards

In [416]:
UK_Employment_Historical_Board.rename(columns={'ED/NED': 'ED/NED/SM'}, inplace=True)
UK_Employment_Current_Board.rename(columns={'ED/NED': 'ED/NED/SM'}, inplace=True)

In [417]:
# Merge all the dataframes related to employees into one
UK_lst = [UK_Employment_Historical_Board, UK_Employment_Current_Board,
          UK_Employment_Historical_NonBoard, UK_Employment_Current_NonBoard]
UK_Employment = pd.concat(UK_lst, ignore_index=True)

### Employment Data cleaning

Check the duplicated rows of the different dataframes

In [418]:
UK_Employment.duplicated().sum()

51421

Remove the duplicates in the dataframes

In [419]:
UK_Employment.drop_duplicates(inplace=True)

### Directors' Characteristics cleaning

In [420]:
UK_Employment_Characteristics = UK_Employment_Characteristics.loc[:, [
    'Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 5_level_0', 'Unnamed: 6_level_0']]
UK_Employment_Characteristics = UK_Employment_Characteristics.droplevel(
    0, axis=1)  # drop the first level of the df


In [421]:
UK_Employment_Characteristics.drop_duplicates(inplace=True, ignore_index=True)


In [422]:
UK_Employment_Characteristics

,Director Name,DirectorID*,Gender,Nationality
0,A (Freddie) O Fletcher,3598506955,M,British
1,A (Joe) Joseph Burnett-Stuart,86082746,M,British
2,A Davies,21996074363,M,Unknown
3,A Dick Boer,52343612106,M,Dutch
4,A Gary Ames,305336778,M,American
...,...,...,...,...
127934,Zuzanna (Suz) Pasierbinska-Wilson,13061926218,F,Unknown
127935,Zuzanna Thompson,194305412032,F,Unknown
127936,Zvi Frank,5441673543,M,Unknown
127937,Zviad Shelia,6457313156,M,Unknown


Merge UK_Employment with UK_Employment_Characteristics

In [423]:
# Merge the dataframes together by DirectorID*
UK_df = pd.merge(UK_Employment, UK_Employment_Characteristics,
                 on='DirectorID*', how='left')

# Drop Director Name_y column
UK_df.drop(['Director Name_y'], axis=1, inplace=True)
UK_df

In [428]:
UK_df

,Country,DirectorID*,Director Name_x,CompanyID*,Company Name,Company ISIN,Company Ticker,Company Index,Sector Name,Company Type,Role,Role Description,ED/NED/SM,Start Date,End Date,Committee Name,Committee Role,Gender,Nationality
0,United Kingdom - England,3598506955,A (Freddie) O Fletcher,275802488,METNOR GROUP PLC (De-listed 04/2009),"GB0003782249, QQ0007112493",MTG,NaN,Steel & Other Metals,Quoted,NED,NaN,NED,2002-04-30 00:00:00,2004-04-01 00:00:00,Audit,Member,M,British
1,United Kingdom - England,3598506955,A (Freddie) O Fletcher,275802488,METNOR GROUP PLC (De-listed 04/2009),"GB0003782249, QQ0007112493",MTG,NaN,Steel & Other Metals,Quoted,NED,NaN,NED,2002-04-30 00:00:00,2004-04-01 00:00:00,Remuneration,Member,M,British
2,United Kingdom - Scotland,86082746,A (Joe) Joseph Burnett-Stuart,15122719,ABERDEEN ASSET MANAGEMENT PLC (Aberdeen Trust ...,GB0000031285,ADN,NaN,Speciality & Other Finance,Quoted,Independent NED,NaN,NED,1991-12-02 00:00:00,2000-01-14 00:00:00,Remuneration,Member,M,British
3,United Kingdom - England,86082746,A (Joe) Joseph Burnett-Stuart,92469182,CALEDONIA INVESTMENTS PLC,GB0001639920,CLDN,FTSE 250,Investment Companies,Quoted,Senior Independent NED,NaN,NED,1990,2002-12-31 00:00:00,Audit,Chairman,M,British
4,United Kingdom - England,86082746,A (Joe) Joseph Burnett-Stuart,92469182,CALEDONIA INVESTMENTS PLC,GB0001639920,CLDN,FTSE 250,Investment Companies,Quoted,Senior Independent NED,NaN,NED,1990,2002-12-31 00:00:00,Nomination,Member,M,British
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376255,United States,6956209411,Zsoka Taylor McDonald,24564197727,AVANGRID INC,US05351W1036,AGR,NaN,Electricity,Quoted,Senior VP/Chief Sustainability Officer,Corporate Communications,SM,2021-05-06 00:00:00,NaN,NaN,NaN,M,Unknown
376256,Jersey,25698088845,Zsuzsa Poós,22959055832,WIZZ AIR HOLDINGS PLC,JE00BN574F90,WIZZ,FTSE 250,Leisure & Hotels,Quoted,Chief Customer Marketing officer,NaN,SM,2020-07-01 00:00:00,NaN,NaN,NaN,F,Hungarian
376257,United Kingdom - England,22365404966,Zura Nemsadze,290514111635,GEORGIA CAPITAL PLC,GB00BF4HYV08,CGEO,FTSE SMALL CAP,Speciality & Other Finance,Quoted,Head of Finance,NaN,SM,2018-05-29 00:00:00,NaN,NaN,NaN,M,Unknown
376258,United Kingdom - England,25433428594,Zurab Kokosadze,189150011651,BANK OF GEORGIA GROUP PLC (BGEO Group PLC prio...,GB00BF4HYT85,BGEO,FTSE SMALL CAP,Banks,Quoted,Deputy CEO - Corporate Banking,NaN,SM,2021-03-10 00:00:00,NaN,NaN,NaN,M,Unknown


## ROW Data

### Load in the Data

In [260]:
# Read all the ROW files
## Director
ROW_Director_Education = pd.read_excel("ROW/ROW - Director Profile - Education & Achievements.xlsx")
ROW_Director_Characteristics = pd.read_excel(
    'ROW/ROW - Director Profile - Characteristics.xlsx', header=[0, 1])
ROW_Director_Employment_Current_Board = pd.read_excel(
    'ROW/ROW - Director Profile - Employment Current Board.xlsx')
ROW_Director_Employment_Current_NonBoard = pd.read_excel(
    'ROW/ROW - Director Profile - Employment Current NonBoard.xlsx')
ROW_Director_Employment_Historical_Board_1 = pd.read_excel(
    'ROW/ROW - Director Profile - Employment Historical Board - 1.xlsx')
ROW_Director_Employment_Historical_Board_2 = pd.read_excel(
    'ROW/ROW - Director Profile - Employment Historical Board - 2.xlsx')
ROW_Director_Employment_Historical_NonBoard = pd.read_excel(
    'ROW/ROW - Director Profile - Employment Historical NonBoard.xlsx')

## SMDEs
ROW_SMDEs_Education = pd.read_excel('ROW/ROW - SMDEs Profile - Education & Achievements.xlsx')
ROW_SMDEs_Characteristics_1 = pd.read_excel(
    'ROW/ROW - SMDEs Profile - Characteristics - 1.xlsx', header=[0, 1])
ROW_SMDEs_Characteristics_2 = pd.read_excel(
    'ROW/ROW - SMDEs Profile - Characteristics - 2.xlsx', header=[0, 1])
ROW_SMDEs_Employment_Current_Board = pd.read_excel(
    'ROW/ROW - SMDEs Profile - Employment Current Board.xlsx')
ROW_SMDEs_Employment_Current_NonBoard = pd.read_excel(
    'ROW/ROW - SMDEs Profile - Employment Current NonBoard.xlsx')
ROW_SMDEs_Employment_Historical_Board = pd.read_excel(
    'ROW/ROW - SMDEs Profile - Employment Historical Board.xlsx')
ROW_SMDEs_Employment_Historical_NonBoard_1 = pd.read_excel(
    'ROW/ROW - SMDEs Profile - Employment Historical NonBoard - 1.xlsx')
ROW_SMDEs_Employment_Historical_NonBoard_2 = pd.read_excel(
    'ROW/ROW - SMDEs Profile - Employment Historical NonBoard - 2.xlsx')


### Stacking the File Multiples

In [430]:
ROW_Director_Employment_Historical_Board = pd.concat(
    [ROW_Director_Employment_Historical_Board_1, ROW_Director_Employment_Historical_Board_2])
ROW_SMDEs_Characteristics = pd.concat(
    [ROW_SMDEs_Characteristics_1, ROW_SMDEs_Characteristics_2])
ROW_SMDEs_Employment_Historical_NonBoard = pd.concat(
    [ROW_SMDEs_Employment_Historical_NonBoard_1, ROW_SMDEs_Employment_Historical_NonBoard_2])


In [431]:
ROW_Education = pd.concat([ROW_Director_Education, ROW_SMDEs_Education])
ROW_Employment_Historical_Board = pd.concat(
    [ROW_Director_Employment_Historical_Board, ROW_SMDEs_Employment_Historical_Board], ignore_index=True)
ROW_Employment_Current_Board = pd.concat(
    [ROW_Director_Employment_Current_Board, ROW_SMDEs_Employment_Current_Board], ignore_index=True)
ROW_Employment_Historical_NonBoard = pd.concat(
    [ROW_Director_Employment_Historical_NonBoard, ROW_SMDEs_Employment_Historical_NonBoard], ignore_index=True)
ROW_Employment_Current_NonBoard = pd.concat(
    [ROW_Director_Employment_Current_NonBoard, ROW_SMDEs_Employment_Current_NonBoard], ignore_index=True)
ROW_Employment_Characteristics = pd.concat(
    [ROW_Director_Characteristics, ROW_SMDEs_Characteristics], ignore_index=True)


Selecting just rows with Ticker: which represents that the company is public (our main scope of investigation)

In [432]:
# Select the companies where the Ticker is not null in all the files -> we just want public listed companies
ROW_Employment_Historical_Board = ROW_Employment_Historical_Board[ROW_Employment_Historical_Board['Company Ticker'].notnull(
)].reset_index(drop=True)
ROW_Employment_Current_Board = ROW_Employment_Current_Board[ROW_Employment_Current_Board['Company Ticker'].notnull(
)].reset_index(drop=True)
ROW_Employment_Historical_NonBoard = ROW_Employment_Historical_NonBoard[
    ROW_Employment_Historical_NonBoard['Company Ticker'].notnull()].reset_index(drop=True)
ROW_Employment_Current_NonBoard = ROW_Employment_Current_NonBoard[ROW_Employment_Current_NonBoard['Company Ticker'].notnull(
)].reset_index(drop=True)


Change the column name of ED/NED of historical and current board dataframes into ED/NED/SM for the concat process afterwards

In [433]:
ROW_Employment_Historical_Board.rename(
    columns={'ED/NED': 'ED/NED/SM'}, inplace=True)
ROW_Employment_Current_Board.rename(
    columns={'ED/NED': 'ED/NED/SM'}, inplace=True)


In [434]:
# Merge all the dataframes related to employees into one
ROW_lst = [ROW_Employment_Historical_Board, ROW_Employment_Current_Board,
          ROW_Employment_Historical_NonBoard, ROW_Employment_Current_NonBoard]
ROW_Employment = pd.concat(ROW_lst, ignore_index=True)


### Employment Data cleaning

In [436]:
ROW_Employment.duplicated().sum()

112631

Remove the duplicates in the dataframes

In [437]:
ROW_Employment.drop_duplicates(inplace=True)

### Directors' Characteristics cleaning

In [438]:
ROW_Employment_Characteristics = ROW_Employment_Characteristics.loc[:, [
    'Unnamed: 0_level_0', 'Unnamed: 1_level_0','Unnamed: 5_level_0', 'Unnamed: 6_level_0']]
ROW_Employment_Characteristics = ROW_Employment_Characteristics.droplevel(
    0, axis=1)  # drop the first level of the df


In [439]:
ROW_Employment_Characteristics.drop_duplicates(inplace=True, ignore_index=True)

In [440]:
ROW_Employment_Characteristics

,Director Name,DirectorID*,Gender,Nationality
0,. (Hanny) Handayani,203855812711,F,Indonesian
1,. (Jojo) Sugiharjo,196505812194,M,Indonesian
2,. Achyat,204188712732,M,Indonesian
3,. Afriwandi,178031310799,M,Unknown
4,. Ahyanizzaman,14559847907,M,Unknown
...,...,...,...,...
239292,Zvika Shamir,6436402635,M,Unknown
239293,Zvika Shenfeld,53536412928,M,Unknown
239294,Zvika Weber,5467924068,M,Unknown
239295,Zweli Manyathi,8775087049,M,Unknown


Merge ROW_Employment with ROW_Employment_Characteristics

In [441]:
# Merge both dataframes together
ROW_df = pd.merge(ROW_Employment, ROW_Employment_Characteristics,
                  on='DirectorID*', how='left')
            
# Drop Director Name_y column
ROW_df.drop(['Director Name_y'], axis=1, inplace=True)
ROW_df


In [443]:
ROW_df


,Country,DirectorID*,Director Name_x,CompanyID*,Company Name,Company ISIN,Company Ticker,Company Index,Sector Name,Company Type,Role,Role Description,ED/NED/SM,Start Date,End Date,Committee Name,Committee Role,Gender,Nationality
0,Indonesia,203855812711,. (Hanny) Handayani,20959501873,PT BANK TABUNGAN NEGARA (PERSERO) TBK (BANK BTN),ID1000113707,BBTN,NaN,Banks,Quoted,Director - Consumer Banking,NaN,ED,2016-09-16 00:00:00,2017-10-18 00:00:00,NaN,NaN,F,Indonesian
1,Indonesia,203855812711,. (Hanny) Handayani,20954951873,PT GARUDA INDONESIA (PERSERO) TBK,ID1000118300,GIAA,NaN,Transport,Quoted,Executive VP - Commercial,MarketingAlso in Sky Team Since 2015,ED,2014-12-12 00:00:00,2016-04-15 00:00:00,NaN,NaN,F,Indonesian
2,Indonesia,196505812194,. (Jojo) Sugiharjo,21456903298,PT JASA MARGA (PERSERO) TBK,ID1000108103,JSMR,NaN,Transport,Quoted,Commissioner,NaN,NED,2017-03-15 00:00:00,2021-05-27 00:00:00,Audit,Member,M,Indonesian
3,Indonesia,204188712732,. Achyat,276712010563,PT TOTALINDO EKA PERSADA TBK,ID1000139207,TOPS,NaN,Construction & Building Materials,Quoted,FD,Director of Accounting and Tax,ED,2017-06-16 00:00:00,2019,NaN,NaN,M,Indonesian
4,Indonesia,204188712732,. Achyat,276712010563,PT TOTALINDO EKA PERSADA TBK,ID1000139207,TOPS,NaN,Construction & Building Materials,Quoted,Commissioner,NaN,NED,2019,2020-03-16 00:00:00,NaN,NaN,M,Indonesian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818319,Israel,13122316295,Zvika (Zvi) Shwimmer,2734288787,ELCO LTD (Elco Holdings Ltd prior to 02/2014),IL0006940345,ELCO,NaN,Engineering & Machinery,Quoted,Division CEO,Electra Consumer Products Ltd,SM,2018-10-02 00:00:00,NaN,NaN,NaN,M,Unknown
818320,South Africa,8775087049,Zweli Manyathi,8305712816,STANDARD BANK GROUP LTD,ZAE000109815,SBK,NaN,Banks,Quoted,Division CEO,Business & Commercial Clients,SM,2021-01-01 00:00:00,NaN,NaN,NaN,M,Unknown
818321,South Africa,24711437875,Zweli Ndese,203388297,HARMONY GOLD MINING CO LTD,ZAE000015228,HAR,NaN,Mining,Quoted,Regional General Manager,Moab Khotsong and Doornkop,SM,2020-10-01 00:00:00,NaN,NaN,NaN,M,Unknown
818322,South Africa,23296106269,Zwelithini (Zweli) Madhlala,24088637209,INDLUPLACE PROPERTIES LTD,ZAE000201125,ILU,NaN,Real Estate,Quoted,Manager,Utilities Manager,SM,2019-03-01 00:00:00,NaN,NaN,NaN,M,Unknown


## Save the regional files

In [ ]:
# Save the dataframe to a csv file
# Europe_df.to_csv("Europe_df.csv", index=False)
# NA_df.to_csv("NA_df.csv", index=False)
# UK_df.to_csv('UK_df.csv', index=False)
# ROW_df.to_csv('ROW_df.csv', index=False)

## Education Dataframe

In [ ]:
# Define a function that will proceed the cleaning process for education dataframes
def cleaning(df):
    df = df.loc[:,['DirectorID*', 'Institution Name', 'Qualification']]
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['Qualification'].fillna('Other', inplace=True)
    df['Category']= ''
    return df

In [ ]:
# Apply the cleaning function to the education dataframes
Europe_Education = cleaning(Europe_Education)
NA_Education= cleaning(NA_Education)
UK_Education = cleaning(UK_Education)
ROW_Education = cleaning(ROW_Education)

In [ ]:
# Concat all the education files together
Education = pd.concat([Europe_Education, NA_Education, UK_Education, ROW_Education])

In [ ]:
# Check duplicates in the Education dataframe
Education.duplicated().sum()

In [ ]:
# Remove duplicates from the Education dataframe
Education.drop_duplicates(inplace=True)

# Reset index
Education.reset_index(drop=True, inplace=True)
Education

In [ ]:
# Save the dataframe to a csv file
#Education.to_csv("Education_dummy.csv", index=False)

### Preprocessing

In [ ]:
# Get rows that contains the strings for each qualification category
bachelor = Education['Qualification'].str.contains('Graduated|BA|BS|BE|BCom|LLB|Bachelor|MD|DMD|AB|Baccalaureate|BTech')
master = Education['Qualification'].str.contains('Masters|MS|MA|JS|Post|MEng|LLM|MPA|MEc')
doctor = Education['Qualification'].str.contains('PhD|Doctor|PharmD|Pharmacy')
mba = Education['Qualification'].str.contains('MBA')
degree = (Education['Qualification'] == 'Degree') | (Education['Qualification'] == 'Degree (Hons)')

In [ ]:
# Get the index of each row that contains the string for each qualification category
bachelor_index = Education[bachelor].index
master_index = Education[master].index
doctor_index = Education[doctor].index
mba_index = Education[mba].index
degree_index = Education[degree].index

In [ ]:
# Fill the Category column with the appropriate qualification category
Education.loc[bachelor_index,'Category'] = 'Bachelor'
Education.loc[master_index,'Category'] = 'Master'
Education.loc[doctor_index,'Category'] = 'Doctorate'
Education.loc[mba_index,'Category'] = 'MBA'
Education.loc[degree_index,'Category'] = 'Bachelor'

# Fill the rest of the rows in Category column with Other
Education['Category'].replace('','Other', inplace=True)

In [ ]:
# Know the counts for each category
Education['Category'].value_counts()

In [ ]:
# Convert the category column to dummy variables
Education = pd.get_dummies(Education, columns=['Category'])

# Rename the column names
Education.rename(columns={'Category_Bachelor': 'Bachelor', 'Category_Master': 'Master', 'Category_Doctorate': 'Doctorate', 'Category_MBA': 'MBA', 'Category_Other':'Other'}, inplace=True)
Education

In [ ]:
# Group by DirectorID*
Education_group = Education.groupby(['DirectorID*']).sum().reset_index()
Education_group

In [ ]:
# Save Education_group to a csv file
#Education_group.to_csv('Education_dummy.csv', index=False)

# Raw dataframe

In [30]:
# Read all the CSV files
# Europe_df = pd.read_csv("Europe_df.csv")
# NA_df = pd.read_csv("NA_df.csv")
# UK_df = pd.read_csv("UK_df.csv")
# ROW_df = pd.read_csv("ROW_df.csv")

<ipython-input-30-98380af579a5>:2: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  Europe_df = pd.read_csv("Europe_df.csv")
<ipython-input-30-98380af579a5>:3: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  NA_df = pd.read_csv("NA_df.csv")
<ipython-input-30-98380af579a5>:4: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  UK_df = pd.read_csv("UK_df.csv")
<ipython-input-30-98380af579a5>:5: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  ROW_df = pd.read_csv("ROW_df.csv")


In [31]:
# Merge all the datasets together 
df_raw = pd.concat([Europe_df, NA_df, UK_df, ROW_df], ignore_index=True)

In [33]:
# Save df as csv
# df_raw.to_csv("Raw_df.csv", index=False)

In [2]:
df_raw= pd.read_csv('Raw_df.csv')

/Users/xiawang/opt/anaconda3/envs/DT/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: DtypeWarning: Columns (14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_raw

,Country,DirectorID*,Director Name_x,CompanyID*,Company Name,Company ISIN,Company Ticker,Company Index,Sector Name,Company Type,Role,Role Description,ED/NED/SM,Start Date,End Date,Committee Name,Committee Role,Gender,Nationality
0,Netherlands,11345263500,A (Ab) J Pasman,17593156,ACCELL GROUP NV,NL0009767532,ACCEL,NaN,Leisure Goods,Quoted,Independent Chairman,NaN,NED,2010-04-22 00:00:00,2019-04-24 00:00:00,Remuneration and Selection,Member,M,Dutch
1,Netherlands,11345263500,A (Ab) J Pasman,17593156,ACCELL GROUP NV,NL0009767532,ACCEL,NaN,Leisure Goods,Quoted,Independent Director,NaN,NED,2019-04-24 00:00:00,2019-12-31 00:00:00,Remuneration and Selection,Member,M,Dutch
2,Austria,51690811635,A (Aris) G Bogdaneris,8489622427,RAIFFEISEN BANK INTERNATIONAL AG (Raiffeisen I...,AT0000606306,RBI,ATX,Banks,Quoted,Board Member - ED,Retail Banking,ED,2004-10-01 00:00:00,2010-12-01 00:00:00,NaN,NaN,M,Canadian
3,Austria,51690811635,A (Aris) G Bogdaneris,8489622427,RAIFFEISEN BANK INTERNATIONAL AG (Raiffeisen I...,AT0000606306,RBI,ATX,Banks,Quoted,COO,Responsible for Consumer Banking Group & Austr...,ED,2010-12-01 00:00:00,2015-03-31 00:00:00,NaN,NaN,M,Canadian
4,Netherlands,3590756873,A (Dries) B Van Luyk,13653543,AALBERTS NV (Aalberts Industries NV prior to 0...,NL0000852564,AALB,AEX MID-CAP,Engineering & Machinery,Quoted,Board Member - SD,NaN,NED,1996,2007-04-23 00:00:00,NaN,NaN,M,Dutch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2809635,Israel,13122316295,Zvika (Zvi) Shwimmer,2734288787,ELCO LTD (Elco Holdings Ltd prior to 02/2014),IL0006940345,ELCO,NaN,Engineering & Machinery,Quoted,Division CEO,Electra Consumer Products Ltd,SM,2018-10-02 00:00:00,NaN,NaN,NaN,M,Unknown
2809636,South Africa,8775087049,Zweli Manyathi,8305712816,STANDARD BANK GROUP LTD,ZAE000109815,SBK,NaN,Banks,Quoted,Division CEO,Business & Commercial Clients,SM,2021-01-01 00:00:00,NaN,NaN,NaN,M,Unknown
2809637,South Africa,24711437875,Zweli Ndese,203388297,HARMONY GOLD MINING CO LTD,ZAE000015228,HAR,NaN,Mining,Quoted,Regional General Manager,Moab Khotsong and Doornkop,SM,2020-10-01 00:00:00,NaN,NaN,NaN,M,Unknown
2809638,South Africa,23296106269,Zwelithini (Zweli) Madhlala,24088637209,INDLUPLACE PROPERTIES LTD,ZAE000201125,ILU,NaN,Real Estate,Quoted,Manager,Utilities Manager,SM,2019-03-01 00:00:00,NaN,NaN,NaN,M,Unknown


# Cleanned Dataframe

In [34]:
# Select the observations that are not NED
df = df_raw[(df_raw['ED/NED/SM'] != 'NED')]

In [35]:
# Remove the individuals without gender information
df.dropna(subset=['Gender'], inplace=True)

In [36]:
# Drop the columns that will not be used for analysis
df.drop(columns=['Role Description','Committee Name', 'Committee Role'], inplace=True)

In [37]:
# Check duplicates in the dataset
df.duplicated().sum()

236208

In [38]:
# Drop duplicates in the dataset
df.drop_duplicates(inplace=True)

In [39]:
# Remove the UnKnown from Start Date
df = df[df['Start Date'] != 'UnKnown']
df = df[df['End Date'] != 'UnKnown']

In [40]:
import numpy as np

In [41]:
# Fill the NA in End date column by the current date
current_date = '2021-11-01'
df['End Date'].fillna(current_date, inplace=True)

In [42]:
# Convert Start date column to a datetime
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['End Date'] = pd.to_datetime(df['End Date'])

In [43]:
# Check if the End Date is after Start Date
df.loc[df['End Date'] < df['Start Date']]

,Country,DirectorID*,Director Name_x,CompanyID*,Company Name,Company ISIN,Company Ticker,Company Index,Sector Name,Company Type,Role,ED/NED/SM,Start Date,End Date,Gender,Nationality
3999,Russian Federation,10655991477,Alexander Grigoryevitch Yastrib,13187226371,BANK OF MOSCOW (BMO) (De-listed 05/2016),RU0006571916,MMBM,NaN,Banks,Quoted,First Executive VP,ED,2011-07-27,2011-01-01,M,Russian
5505,Poland,196426612186,Alicja Barbara Klimiuk,195582812128,ENERGA SA,PLENERG00022,ENG,NaN,Electricity,Quoted,Chairman,ED,2019-06-05,2019-01-01,F,Unknown
7770,Sweden,13941747254,Andreas Bo Philipson,4997312907,CATENA AB,SE0001664707,CATE,NaN,Transport,Quoted,CEO,ED,2013-09-23,2013-01-01,M,Unknown
13647,Norway,3489275609,Arne Barmen,314017619,MOWI ASA (Marine Harvest ASA prior to 01/2019),NO0003054108,MOWI,"EURONEXT 100, OBX",Food Producers & Processors,Quoted,FD,ED,2003-01-10,2003-01-01,M,Swedish
20881,United States,167984673,Bruno Bonnell,222639793,ATARI INC (Infogrames prior to 03/2003) (De-li...,"US04651M1053, US04651M2044, US45665T1079",ATAR,NaN,Software & Computer Services,Quoted,Chairman/Interim CEO,ED,2005-06-06,2005-01-01,M,French
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2736201,Israel,8891888277,Yaron Naos,9891188001,PROTALIX BIOTHERAPEUTICS INC (Orthodontix Inc ...,"US74365A1016, US74365A3095",PLX,NaN,Pharmaceuticals and Biotechnology,Quoted,Vice President - Ops,SM,2008-05-06,2008-01-01,M,Unknown
2736338,China,24247497389,Ye Qi,14295287630,CHINA EVERBRIGHT BANK CO LTD,"CNE100000SL4, CNE100001QW3","601818, 6818",NaN,Banks,Quoted,VP/Chief Business Officer,SM,2020-07-24,2020-07-01,F,Unknown
2737179,Japan,8895118307,Yuki Ohshima,310537288,ORIX CORP,JP3200450009,8591,NaN,Speciality & Other Finance,Quoted,Advisor,SM,2015-06-23,2015-01-01,M,Unknown
2737370,Taiwan Territory of,25805558949,Yu-Yin Ou,22755285552,KING'S TOWN BANK CO LTD,"TW00001010T9, TW0002809001","01010T, 2809",NaN,Banks,Quoted,Regional Director,SM,2020-02-25,2020-01-01,M,Chinese


In [44]:
# Remove the observations that have an End Date before the Start Date
df = df[df['End Date'] >= df['Start Date']]

In [45]:
df.reset_index(drop=True, inplace=True)

In [46]:
df

,Country,DirectorID*,Director Name_x,CompanyID*,Company Name,Company ISIN,Company Ticker,Company Index,Sector Name,Company Type,Role,ED/NED/SM,Start Date,End Date,Gender,Nationality
0,Austria,51690811635,A (Aris) G Bogdaneris,8489622427,RAIFFEISEN BANK INTERNATIONAL AG (Raiffeisen I...,AT0000606306,RBI,ATX,Banks,Quoted,Board Member - ED,ED,2004-10-01,2010-12-01,M,Canadian
1,Austria,51690811635,A (Aris) G Bogdaneris,8489622427,RAIFFEISEN BANK INTERNATIONAL AG (Raiffeisen I...,AT0000606306,RBI,ATX,Banks,Quoted,COO,ED,2010-12-01,2015-03-31,M,Canadian
2,Netherlands,6603475665,A (Jos) Andeweg,407445259,UNIT4 NV (Unit 4 Agresso NV prior to 06/2010) ...,NL0000389096,UNIT4,NaN,Software & Computer Services,Quoted,Board Member - ED,ED,2007-01-01,2014-01-01,M,Dutch
3,Netherlands,16531779758,A (Tineke) J Veldhuis-Hagedoorn,210628883,HOLLAND COLOURS NV,NL0000440311,HOLCO,NaN,Chemicals,Quoted,Regional MD,ED,1979-01-01,2016-01-01,F,Dutch
4,Netherlands,6446592907,A (Ton) G Hillen,206138525,HEIJMANS NV,NL0009269109,HEIJM,NaN,Construction & Building Materials,Quoted,Board Member - ED,ED,2012-04-18,2016-12-01,M,Dutch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1322657,Israel,23236976192,Zvika (Zvi) Fishheimer,3823112661,SHUFERSAL LTD,IL0007770378,SAE,NaN,Food & Drug Retailers,Quoted,Executive VP - HR/Admin,SM,2015-02-10,2021-11-01,M,Unknown
1322658,Israel,13122316295,Zvika (Zvi) Shwimmer,2734288787,ELCO LTD (Elco Holdings Ltd prior to 02/2014),IL0006940345,ELCO,NaN,Engineering & Machinery,Quoted,Division CEO,SM,2018-10-02,2021-11-01,M,Unknown
1322659,South Africa,8775087049,Zweli Manyathi,8305712816,STANDARD BANK GROUP LTD,ZAE000109815,SBK,NaN,Banks,Quoted,Division CEO,SM,2021-01-01,2021-11-01,M,Unknown
1322660,South Africa,24711437875,Zweli Ndese,203388297,HARMONY GOLD MINING CO LTD,ZAE000015228,HAR,NaN,Mining,Quoted,Regional General Manager,SM,2020-10-01,2021-11-01,M,Unknown


In [47]:
# Save the df to a csv file
#df.to_csv("Clean_df.csv", index=False) 

# Creation of Role Dataframe

### Cleaning and Preprocessing

In [ ]:
# Read the df from the csv file
df = pd.read_csv("Clean_df.csv")

In [21]:
# Create a new dataframe from the df
role = df[['DirectorID*', 'Director Name_x','CompanyID*', 'Company Name','Sector Name' ,'Role','Start Date', 'End Date']]

In [22]:
# Check duplicates in the role dataframe
role.duplicated().sum()

18

In [23]:
# Remove duplicates
role.drop_duplicates(inplace=True)

<ipython-input-23-cc180f6c3a44>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role.drop_duplicates(inplace=True)


In [24]:
# Sort the dataframe by Start Date and DirectorID*
role.sort_values(by=['DirectorID*', 'Start Date'], inplace=True)
role=role.reset_index(drop=True)

<ipython-input-24-2e748d299bea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role.sort_values(by=['DirectorID*', 'Start Date'], inplace=True)


In [25]:
# Create new columns: Next Role, Experience, Source Company and Source company sector
role ['Next Role'] = ''
role['Experience'] = role['End Date'] - role['Start Date']
role['Source company'] = ''
role['Source company sector'] = ''

# Convert the Experience column to years
role['Experience'] = round(role['Experience'].dt.days / 365, 2)

In [28]:
# Merge the education dataframe with role dataframe
role = pd.merge(role, Education, on='DirectorID*', how='left')
role

,DirectorID*,Director Name_x,CompanyID*,Company Name,Sector Name,Role,Start Date,End Date,Next Role,Experience,Source company,Source company sector,Bachelor,Doctorate,MBA,Master,Other
0,1680,David John Blakeman,2607112325,LOOKERS PLC,Automobiles & Parts,Various Positions,1984-01-01,1989-01-01,,5.01,,,1.0,0.0,0.0,0.0,0.0
1,1680,David John Blakeman,2607112325,LOOKERS PLC,Automobiles & Parts,ED/Secretary,1989-01-01,2007-01-02,,18.01,,,1.0,0.0,0.0,0.0,0.0
2,2440,Andrew David McClure,3668011673,SMART(J.)& CO(CONTRACTORS) PLC,Real Estate,Various Positions,1964-01-01,1985-01-01,,21.02,,,NaN,NaN,NaN,NaN,NaN
3,2440,Andrew David McClure,3668011673,SMART(J.)& CO(CONTRACTORS) PLC,Real Estate,Company Secretary,1985-01-01,1987-01-01,,2.00,,,NaN,NaN,NaN,NaN,NaN
4,2440,Andrew David McClure,3668011673,SMART(J.)& CO(CONTRACTORS) PLC,Real Estate,FD/Secretary,1987-01-01,2011-11-18,,24.90,,,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1322639,207076712928,Kenneth (Ken) Feather,175498510596,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,Various Positions,2010-10-01,2013-12-01,,3.17,,,1.0,0.0,0.0,0.0,1.0
1322640,207076712928,Kenneth (Ken) Feather,175498510596,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,Division VP,2013-12-01,2015-11-01,,1.92,,,1.0,0.0,0.0,0.0,1.0
1322641,207076712928,Kenneth (Ken) Feather,175498510596,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,Division VP - Sales/Marketing,2015-12-01,2016-08-01,,0.67,,,1.0,0.0,0.0,0.0,1.0
1322642,207076912928,Olivier Bojarski,91518506,BELDEN INC (Belden CDT Inc prior to 05/2007),Electronic & Electrical Equipment,Executive VP,2021-05-01,2021-11-01,,0.50,,,1.0,0.0,1.0,0.0,0.0


In [ ]:
# For loop to fill the new columns created above
for index, row in role.iterrows():
    if index < len(role)-1:
        # If the next directorID is the same
        if role.iloc[index,0] == role.iloc[index+1,0]:
            # The new role is the same as the next row of role
            role.iloc[index, 8] = role.iloc[index+1, 5]
            # Accumulative experience (sum the previous experience with the current one)
            role.iloc[index+1, 9] = round(role.iloc[index, 9] + role.iloc[index+1,9],2)
            # The source company is the same as the previous row
            role.iloc[index+1, 10] = role.iloc[index, 3]
            # The source company sector is the same as the previous row
            role.iloc[index+1, 11] = role.iloc[index, 4]
        else:
            pass
    else:
        pass

In [ ]:
# Remove the observations with the same start date and End Date
role = role[role['Start Date'] != role['End Date']]

# Add a new column for future purposes
role['Current Role'] = role['Role']

# Reorder the columns for clearer view
role= role[['DirectorID*', 'Director Name_x', 'CompanyID*', 'Company Name',
       'Sector Name', 'Role','Current Role', 'Start Date', 'End Date',
       'Experience', 'Next Role','Source company', 'Source company sector', 'Bachelor',
       'Doctorate', 'MBA', 'Master', 'Other']]

In [ ]:
# Save role dataframe to csv
#role.to_csv('Role.csv', index=False)

In [ ]:
# Count the frequency of the roles
frequency = role[['Current Role','Next Role']].value_counts().reset_index(name='Count')
frequency

# Master dataframe

In [3]:
df = pd.read_csv('Clean_df.csv')

In [6]:
df

,Country,DirectorID*,Director Name_x,CompanyID*,Company Name,Company ISIN,Company Ticker,Company Index,Sector Name,Company Type,Role,ED/NED/SM,Start Date,End Date,Gender,Nationality
0,Austria,51690811635,A (Aris) G Bogdaneris,8489622427,RAIFFEISEN BANK INTERNATIONAL AG (Raiffeisen I...,AT0000606306,RBI,ATX,Banks,Quoted,Board Member - ED,ED,2004-10-01,2010-12-01,M,Canadian
1,Austria,51690811635,A (Aris) G Bogdaneris,8489622427,RAIFFEISEN BANK INTERNATIONAL AG (Raiffeisen I...,AT0000606306,RBI,ATX,Banks,Quoted,COO,ED,2010-12-01,2015-03-31,M,Canadian
2,Netherlands,6603475665,A (Jos) Andeweg,407445259,UNIT4 NV (Unit 4 Agresso NV prior to 06/2010) ...,NL0000389096,UNIT4,NaN,Software & Computer Services,Quoted,Board Member - ED,ED,2007-01-01,2014-01-01,M,Dutch
3,Netherlands,16531779758,A (Tineke) J Veldhuis-Hagedoorn,210628883,HOLLAND COLOURS NV,NL0000440311,HOLCO,NaN,Chemicals,Quoted,Regional MD,ED,1979-01-01,2016-01-01,F,Dutch
4,Netherlands,6446592907,A (Ton) G Hillen,206138525,HEIJMANS NV,NL0009269109,HEIJM,NaN,Construction & Building Materials,Quoted,Board Member - ED,ED,2012-04-18,2016-12-01,M,Dutch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1322657,Israel,23236976192,Zvika (Zvi) Fishheimer,3823112661,SHUFERSAL LTD,IL0007770378,SAE,NaN,Food & Drug Retailers,Quoted,Executive VP - HR/Admin,SM,2015-02-10,2021-11-01,M,Unknown
1322658,Israel,13122316295,Zvika (Zvi) Shwimmer,2734288787,ELCO LTD (Elco Holdings Ltd prior to 02/2014),IL0006940345,ELCO,NaN,Engineering & Machinery,Quoted,Division CEO,SM,2018-10-02,2021-11-01,M,Unknown
1322659,South Africa,8775087049,Zweli Manyathi,8305712816,STANDARD BANK GROUP LTD,ZAE000109815,SBK,NaN,Banks,Quoted,Division CEO,SM,2021-01-01,2021-11-01,M,Unknown
1322660,South Africa,24711437875,Zweli Ndese,203388297,HARMONY GOLD MINING CO LTD,ZAE000015228,HAR,NaN,Mining,Quoted,Regional General Manager,SM,2020-10-01,2021-11-01,M,Unknown


In [4]:
role = pd.read_csv('Role.csv')

In [7]:
role

,DirectorID*,Director Name_x,CompanyID*,Company Name,Sector Name,Role,Current Role,Start Date,End Date,Experience,Next Role,Source company,Source company sector,Bachelor,Doctorate,MBA,Master,Other
0,1680,David John Blakeman,2607112325,LOOKERS PLC,Automobiles & Parts,Various Positions,Various Positions,1984-01-01,1989-01-01,5.01,ED/Secretary,NaN,NaN,1.0,0.0,0.0,0.0,0.0
1,1680,David John Blakeman,2607112325,LOOKERS PLC,Automobiles & Parts,ED/Secretary,ED/Secretary,1989-01-01,2007-01-02,23.02,NaN,LOOKERS PLC,Automobiles & Parts,1.0,0.0,0.0,0.0,0.0
2,2440,Andrew David McClure,3668011673,SMART(J.)& CO(CONTRACTORS) PLC,Real Estate,Various Positions,Various Positions,1964-01-01,1985-01-01,21.02,Company Secretary,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2440,Andrew David McClure,3668011673,SMART(J.)& CO(CONTRACTORS) PLC,Real Estate,Company Secretary,Company Secretary,1985-01-01,1987-01-01,23.02,FD/Secretary,SMART(J.)& CO(CONTRACTORS) PLC,Real Estate,NaN,NaN,NaN,NaN,NaN
4,2440,Andrew David McClure,3668011673,SMART(J.)& CO(CONTRACTORS) PLC,Real Estate,FD/Secretary,FD/Secretary,1987-01-01,2011-11-18,47.92,FD,SMART(J.)& CO(CONTRACTORS) PLC,Real Estate,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319337,207076712928,Kenneth (Ken) Feather,175498510596,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,Various Positions,Various Positions,2010-10-01,2013-12-01,19.76,Division VP,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,1.0,0.0,0.0,0.0,1.0
1319338,207076712928,Kenneth (Ken) Feather,175498510596,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,Division VP,Division VP,2013-12-01,2015-11-01,21.68,Division VP - Sales/Marketing,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,1.0,0.0,0.0,0.0,1.0
1319339,207076712928,Kenneth (Ken) Feather,175498510596,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,Division VP - Sales/Marketing,Division VP - Sales/Marketing,2015-12-01,2016-08-01,22.35,NaN,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,1.0,0.0,0.0,0.0,1.0
1319340,207076912928,Olivier Bojarski,91518506,BELDEN INC (Belden CDT Inc prior to 05/2007),Electronic & Electrical Equipment,Executive VP,Executive VP,2021-05-01,2021-11-01,0.50,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0


In [5]:
# Select the columns with the needed information regarding each vacancy and company level
vacancy = df[['End Date','Role','CompanyID*', 'Country','Sector Name', 'Gender']]
# Rename the Gender column in master_df as Ex Role Gender
vacancy.rename(columns={'Gender':'Ex role gender'}, inplace=True)
vacancy

<ipython-input-5-10af4002619f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacancy.rename(columns={'Gender':'Ex role gender'}, inplace=True)


,End Date,Role,CompanyID*,Country,Sector Name,Ex role gender
0,2010-12-01,Board Member - ED,8489622427,Austria,Banks,M
1,2015-03-31,COO,8489622427,Austria,Banks,M
2,2014-01-01,Board Member - ED,407445259,Netherlands,Software & Computer Services,M
3,2016-01-01,Regional MD,210628883,Netherlands,Chemicals,F
4,2016-12-01,Board Member - ED,206138525,Netherlands,Construction & Building Materials,M
...,...,...,...,...,...,...
1322657,2021-11-01,Executive VP - HR/Admin,3823112661,Israel,Food & Drug Retailers,M
1322658,2021-11-01,Division CEO,2734288787,Israel,Engineering & Machinery,M
1322659,2021-11-01,Division CEO,8305712816,South Africa,Banks,M
1322660,2021-11-01,Regional General Manager,203388297,South Africa,Mining,M


In [6]:
# Remove duplicates
vacancy.drop_duplicates(inplace=True)

<ipython-input-6-7b11f3c8a4d7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacancy.drop_duplicates(inplace=True)


In [7]:
# Just select the vacancies after 2000-01-01
vacancy = vacancy[vacancy['End Date'] >= '2000-01-01']

# Sort the dataframe by the End Date
vacancy.sort_values(by=['End Date'], inplace=True)

# Reset the index
vacancy.reset_index(drop=True, inplace=True)
vacancy

<ipython-input-7-263741754e03>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacancy.sort_values(by=['End Date'], inplace=True)


,End Date,Role,CompanyID*,Country,Sector Name,Ex role gender
0,2000-01-01,Vice President,3811412590,United States,Health,M
1,2000-01-01,Senior Manager,334729383,Netherlands,Pharmaceuticals and Biotechnology,M
2,2000-01-01,Leader,186896813,United States,Diversified Industrials,F
3,2000-01-01,Executive VP/COO,1463310710,United States,Chemicals,M
4,2000-01-01,Board Member - ED,161913774,Germany,Insurance,M
...,...,...,...,...,...,...
1114793,2021-11-01,President,10646261426,Hong Kong SAR,Oil & Gas,M
1114794,2021-11-01,Chairman,186603111460,Hong Kong SAR,Speciality & Other Finance,M
1114795,2021-11-01,ED,180483610991,China,Chemicals,M
1114796,2021-11-01,Deputy General Manager,172259410340,Hong Kong SAR,General Retailers,M


In [8]:
vacancy['Role'].value_counts()

Various Positions                                                        51823
CFO                                                                      24158
CEO                                                                      20762
President/CEO                                                            16669
Vice President                                                           16259
                                                                         ...  
Vice Chairman/Division Global Head                                           1
Group Executive/Division Deputy President                                    1
Co-Deputy Chief Investment Officer                                           1
First Senior VP/Chief Administrative Officer/Chief Compliance Officer        1
First Vice Chairman/President/CEO                                            1
Name: Role, Length: 26155, dtype: int64

In [9]:
vacancy.loc[(vacancy['Role']=='CEO')| (vacancy['Role'].str.contains('/CEO'))]

,End Date,Role,CompanyID*,Country,Sector Name,Ex role gender
12,2000-01-01,President/CEO,205388466,United States,Health,M
73,2000-01-01,Chairman/CEO,968311884,Switzerland,Electronic & Electrical Equipment,M
128,2000-01-01,President/CEO,2369710799,United States,Business Services,M
246,2000-01-01,President/CEO,116477466,United States,Pharmaceuticals and Biotechnology,M
295,2000-01-01,Chairman/President/CEO,15034536,United States,Telecommunication Services,M
...,...,...,...,...,...,...
1114763,2021-11-01,Chairwoman/CEO,33471876295,Hong Kong SAR,Telecommunication Services,F
1114764,2021-11-01,CEO,34070777014,China,Consumer Services,M
1114766,2021-11-01,Chairman (Executive)/CEO,180504210999,China,Clothing & Personal Products,M
1114783,2021-11-01,CEO,199134712375,China,Food Producers & Processors,M


## CEO vacancy

In [17]:
# Select just the rows with Role= CEO
CEO_vacancy = vacancy.loc[(vacancy['Role'] == 'CEO') | (vacancy['Role'].str.contains('/CEO'))]

# Replace the Role values to CEO 
CEO_vacancy['Vacancy role'] = 'CEO'

# Remove duplicates
CEO_vacancy.drop_duplicates(inplace=True)

# Split the dataframe into 2 dataframes: 1st decade and 2nd decade
CEO_vacancy_first_decade = CEO_vacancy[CEO_vacancy['End Date'] < '2010-01-01']
CEO_vacancy_second_decade = CEO_vacancy[(CEO_vacancy['End Date'] >= '2010-01-01') & (CEO_vacancy['End Date'] < '2020-01-01')]
CEO_vacancy_last = CEO_vacancy[CEO_vacancy['End Date'] >= '2020-01-01']

# Reset the index
CEO_vacancy_first_decade= CEO_vacancy_first_decade.reset_index(drop=True)
CEO_vacancy_second_decade= CEO_vacancy_second_decade.reset_index(drop=True)
CEO_vacancy_last= CEO_vacancy_last.reset_index(drop=True)

# Convert the End Date column to datetime
CEO_vacancy_first_decade['End Date'] = pd.to_datetime(CEO_vacancy_first_decade['End Date'])
CEO_vacancy_second_decade['End Date'] = pd.to_datetime(CEO_vacancy_second_decade['End Date'])
CEO_vacancy_last['End Date'] = pd.to_datetime(CEO_vacancy_last['End Date'])

<ipython-input-17-0758d1cde3a7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CEO_vacancy['Vacancy role'] = 'CEO'
<ipython-input-17-0758d1cde3a7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CEO_vacancy.drop_duplicates(inplace=True)


In [19]:
CEO_vacancy_first_decade

,End Date,Role,CompanyID*,Country,Sector Name,Ex role gender,Vacancy role
0,2000-01-01,President/CEO,205388466,United States,Health,M,CEO
1,2000-01-01,Chairman/CEO,968311884,Switzerland,Electronic & Electrical Equipment,M,CEO
2,2000-01-01,President/CEO,2369710799,United States,Business Services,M,CEO
3,2000-01-01,President/CEO,116477466,United States,Pharmaceuticals and Biotechnology,M,CEO
4,2000-01-01,Chairman/President/CEO,15034536,United States,Telecommunication Services,M,CEO
...,...,...,...,...,...,...,...
16816,2009-12-31,President/CEO,10700381664,United States,Electronic & Electrical Equipment,M,CEO
16817,2009-12-31,President/CEO,9862062,United States,Banks,M,CEO
16818,2009-12-31,CEO,342679992,United Kingdom - England,Containers & Packaging,M,CEO
16819,2009-12-31,Chairman/President/CEO,849624886,Canada,Pharmaceuticals and Biotechnology,M,CEO


In [20]:
CEO_vacancy_second_decade

,End Date,Role,CompanyID*,Country,Sector Name,Ex role gender,Vacancy role
0,2010-01-01,Regional President/CEO,1106712281,United States,Information Technology Hardware,M,CEO
1,2010-01-01,President/CEO,10038409392,Tanzania,Oil & Gas,M,CEO
2,2010-01-01,Chairman/CEO,974912252,Mexico,Telecommunication Services,M,CEO
3,2010-01-01,Chairman/CEO,1464910815,United States,Engineering & Machinery,M,CEO
4,2010-01-01,Division Co-Chairman/CEO,180006113,Sweden,Software & Computer Services,M,CEO
...,...,...,...,...,...,...,...
26264,2019-12-31,CEO,176920110710,Singapore,Real Estate,M,CEO
26265,2019-12-31,CEO,9790156920,Australia,Mining,M,CEO
26266,2019-12-31,President/CEO,281288510928,United States,Leisure & Hotels,M,CEO
26267,2019-12-31,Executive President/CEO,22004134380,Mexico,Food Producers & Processors,M,CEO


In [21]:
CEO_vacancy_last

,End Date,Role,CompanyID*,Country,Sector Name,Ex role gender,Vacancy role
0,2020-01-01,President/CEO,429777705,United States,Banks,M,CEO
1,2020-01-01,President/CEO,803010069,United States,Electronic & Electrical Equipment,M,CEO
2,2020-01-01,CEO,196874712216,South Korea,Electronic & Electrical Equipment,M,CEO
3,2020-01-01,Division President/CEO,167144555,United States,Utilities - Other,F,CEO
4,2020-01-01,Chairman/CEO,25337388506,Italy,Health,M,CEO
...,...,...,...,...,...,...,...
18220,2021-11-01,Chairwoman/CEO,33471876295,Hong Kong SAR,Telecommunication Services,F,CEO
18221,2021-11-01,CEO,34070777014,China,Consumer Services,M,CEO
18222,2021-11-01,Chairman (Executive)/CEO,180504210999,China,Clothing & Personal Products,M,CEO
18223,2021-11-01,CEO,199134712375,China,Food Producers & Processors,M,CEO


In [14]:
CEO_vacancy['Role'].value_counts()

CEO                                                                    20762
President/CEO                                                          16669
Chairman/CEO                                                            9606
Chairman/President/CEO                                                  5144
Division President/CEO                                                  1119
                                                                       ...  
Chairman/CEO/Chief Risk Officer                                            1
Chairman/CEO/GFD                                                           1
Chairman/CEO/CFO/COO                                                       1
Chairman/CEO/Chief Export Control Officer (Representative Director)        1
First Vice Chairman/President/CEO                                          1
Name: Role, Length: 482, dtype: int64

In [22]:
# Select the Start Date after 2000 in role dataframe
role_sample = role.loc[role['Start Date'] >= '2000-01-01']

# Convert Start Date to datetime format
role_sample['Start Date'] = pd.to_datetime(role_sample['Start Date'])
role_sample['End Date'] = pd.to_datetime(role_sample['End Date'])


# Reset index
role_sample=role_sample.reset_index(drop=True)
role_sample

<ipython-input-22-af0ffc54bf6c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_sample['Start Date'] = pd.to_datetime(role_sample['Start Date'])
<ipython-input-22-af0ffc54bf6c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_sample['End Date'] = pd.to_datetime(role_sample['End Date'])


,DirectorID*,Director Name_x,CompanyID*,Company Name,Sector Name,Role,Current Role,Start Date,End Date,Experience,Next Role,Source company,Source company sector,Bachelor,Doctorate,MBA,Master,Other
0,2440,Andrew David McClure,3668011673,SMART(J.)& CO(CONTRACTORS) PLC,Real Estate,FD,FD,2011-11-18,2011-12-19,48.00,NaN,SMART(J.)& CO(CONTRACTORS) PLC,Real Estate,NaN,NaN,NaN,NaN,NaN
1,3040,Harold (Harry) William Bailey,547611620,ASSOCIATED BRITISH FOODS PLC,Food Producers & Processors,Chairman (Executive),Chairman (Executive),2000-05-31,2002-12-05,23.95,NaN,ASSOCIATED BRITISH FOODS PLC,Food Producers & Processors,0.0,0.0,0.0,0.0,1.0
2,4480,Christophe Aguiton,180106127,ORANGE (France Telecom SA prior to 07/2013),Telecommunication Services,Researcher,Researcher,2001-01-01,2021-11-01,20.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4890,Guido Angiolini,151631319,EDISON SPA (De-listed 09/2012),Utilities - Other,Co-CEO,Co-CEO,2002-12-01,2003-01-22,0.14,Chairman,NaN,NaN,1.0,0.0,0.0,0.0,1.0
4,4890,Guido Angiolini,186516778,GEMINA SPA (Formerly known as Compagnia Genera...,Transport,Chairman,Chairman,2007-05-07,2010-04-28,3.12,MD,EDISON SPA (De-listed 09/2012),Utilities - Other,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104425,207076712928,Kenneth (Ken) Feather,175498510596,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,Various Positions,Various Positions,2010-10-01,2013-12-01,19.76,Division VP,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,1.0,0.0,0.0,0.0,1.0
1104426,207076712928,Kenneth (Ken) Feather,175498510596,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,Division VP,Division VP,2013-12-01,2015-11-01,21.68,Division VP - Sales/Marketing,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,1.0,0.0,0.0,0.0,1.0
1104427,207076712928,Kenneth (Ken) Feather,175498510596,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,Division VP - Sales/Marketing,Division VP - Sales/Marketing,2015-12-01,2016-08-01,22.35,NaN,ARCHER LTD (Seawell Ltd prior to 05/2011),Oil & Gas,1.0,0.0,0.0,0.0,1.0
1104428,207076912928,Olivier Bojarski,91518506,BELDEN INC (Belden CDT Inc prior to 05/2007),Electronic & Electrical Equipment,Executive VP,Executive VP,2021-05-01,2021-11-01,0.50,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0


In [27]:
role_sample.loc[(role_sample['Current Role']== 'CEO') | (role_sample['Current Role'].str.contains('/CEO'))]

,DirectorID*,Director Name_x,CompanyID*,Company Name,Sector Name,Role,Current Role,Start Date,End Date,Experience,Next Role,Source company,Source company sector,Bachelor,Doctorate,MBA,Master,Other
8,57410,Martin John Bettington,8832007673,BIFFA PLC (De-listed 04/2008),Business Services,CEO,CEO,2006-10-09,2008-03-31,13.40,NaN,SEVERN TRENT PLC,Utilities - Other,1.0,0.0,1.0,0.0,0.0
9,85390,Doctor Björn (Nalle) Arne Wahlroos,3516110637,SAMPO OYJ (Formerly Known as Sampo-Leonia Insu...,Banks,Group President/CEO,Group President/CEO,2001-04-05,2009-04-07,8.01,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0
21,381849,Eli Reifman,156132663,B.S.D. CROWN LTD (Emblaze Ltd prior to 08/2014...,Software & Computer Services,CEO,CEO,2000-09-01,2006-12-11,6.28,President,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,386750,Luis Fernando Del Rivero Asensio,412325873,SACYR SA (Sacyr Vallehermoso SA prior to 08/2013),Construction & Building Materials,CEO,CEO,2003-04-30,2003-06-05,6.55,First Vice Chairman/CEO,SACYR SA (Sacyr Vallehermoso SA prior to 08/2013),Construction & Building Materials,2.0,0.0,0.0,0.0,0.0
30,386750,Luis Fernando Del Rivero Asensio,412325873,SACYR SA (Sacyr Vallehermoso SA prior to 08/2013),Construction & Building Materials,First Vice Chairman/CEO,First Vice Chairman/CEO,2003-06-05,2004-11-10,7.99,Chairman (Executive),SACYR SA (Sacyr Vallehermoso SA prior to 08/2013),Construction & Building Materials,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104281,207014412928,Stephen John Bowhill,25723438873,RANGE INTERNATIONAL LTD,Chemicals,CEO,CEO,2020-04-14,2021-11-01,3.98,NaN,RANGE INTERNATIONAL LTD,Chemicals,1.0,0.0,0.0,0.0,0.0
1104386,207067312928,Benoît Gascon,200584012483,MASON GRAPHITE INC (POCML 1 Inc prior to 10/2012),Mining,President/CEO,President/CEO,2012-10-15,2020-03-31,7.46,NaN,NaN,NaN,1.0,0.0,0.0,0.0,2.0
1104388,207068212928,Paavo Nõgene,13548596801,TALLINK GROUP AS,Transport,Chairman/CEO,Chairman/CEO,2018-05-01,2021-11-01,3.51,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0
1104393,207070312928,Mark Pomeranz,286129611298,MOTUS GI HOLDINGS INC,Health,President/CEO,President/CEO,2018-02-14,2018-10-01,0.63,President/COO,NaN,NaN,1.0,0.0,0.0,1.0,0.0


In [27]:
from tqdm import tqdm

In [28]:
CEO_vacancy_first_decade

,End Date,Role,CompanyID*,Country,Sector Name,Ex role gender,Vacancy role
0,2000-01-01,President/CEO,205388466,United States,Health,M,CEO
1,2000-01-01,Chairman/CEO,968311884,Switzerland,Electronic & Electrical Equipment,M,CEO
2,2000-01-01,President/CEO,2369710799,United States,Business Services,M,CEO
3,2000-01-01,President/CEO,116477466,United States,Pharmaceuticals and Biotechnology,M,CEO
4,2000-01-01,Chairman/President/CEO,15034536,United States,Telecommunication Services,M,CEO
...,...,...,...,...,...,...,...
16816,2009-12-31,President/CEO,10700381664,United States,Electronic & Electrical Equipment,M,CEO
16817,2009-12-31,President/CEO,9862062,United States,Banks,M,CEO
16818,2009-12-31,CEO,342679992,United Kingdom - England,Containers & Packaging,M,CEO
16819,2009-12-31,Chairman/President/CEO,849624886,Canada,Pharmaceuticals and Biotechnology,M,CEO


In [29]:
column_list= list(CEO_vacancy.columns)
df_CEO_first = pd.DataFrame()

for index, row in tqdm(CEO_vacancy_first_decade.iterrows()):
    candidates = role_sample.loc[role_sample['Start Date'] >= CEO_vacancy_first_decade['End Date'].iloc[index]]
    position = candidates.loc[(candidates['Role']=='CEO')| (candidates['Role'].str.contains('/CEO'))]
    position[column_list] = CEO_vacancy_first_decade.loc[index,column_list]
    df_CEO_first = df_CEO_first.append(position)

df_CEO_first

0it [00:00, ?it/s]<ipython-input-29-77600b08d529>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  position[column_list] = CEO_vacancy_first_decade.loc[index,column_list]
<ipython-input-29-77600b08d529>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  position[column_list] = CEO_vacancy_first_decade.loc[index,column_list]
<ipython-input-29-77600b08d529>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [32]:
CEO_vacancy_first_decade

,End Date,Role,CompanyID*,Country,Sector Name,Ex role gender,Vacancy role
0,2000-01-01,President/CEO,205388466,United States,Health,M,CEO
1,2000-01-01,Chairman/CEO,968311884,Switzerland,Electronic & Electrical Equipment,M,CEO
2,2000-01-01,President/CEO,2369710799,United States,Business Services,M,CEO
3,2000-01-01,President/CEO,116477466,United States,Pharmaceuticals and Biotechnology,M,CEO
4,2000-01-01,Chairman/President/CEO,15034536,United States,Telecommunication Services,M,CEO
...,...,...,...,...,...,...,...
16816,2009-12-31,President/CEO,10700381664,United States,Electronic & Electrical Equipment,M,CEO
16817,2009-12-31,President/CEO,9862062,United States,Banks,M,CEO
16818,2009-12-31,CEO,342679992,United Kingdom - England,Containers & Packaging,M,CEO
16819,2009-12-31,Chairman/President/CEO,849624886,Canada,Pharmaceuticals and Biotechnology,M,CEO


In [31]:
role_sample.loc[role_sample['Start Date'] >= CEO_vacancy_first_decade['End Date']]

ValueError: Can only compare identically-labeled Series objects

In [1]:
df_CEO

NameError: name 'df_CEO' is not defined

In [ ]:
# Change the order of the columns
dfm_try = df_CEO[['End Date', 'Role', 'CompanyID*', 'Sector Name', 'Ex role gender', 'DirectorID*',
'Director Name_x', 'Gender', 'Current Role','Start Date',]]
# Change the name of End Date column for merging purpose
dfm_try.rename(columns={'End Date':'Date'}, inplace=True)
dfm_try= dfm_try.reset_index(drop=True)
dfm_try